In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [5]:
races = pd.read_csv('https://raw.githubusercontent.com/pavleke/project-formula-1/main/races.csv')
all_races = []
all_weather_main = []
all_weather = []
all_wind = []
wet = ['rain', 'drizzle', 'wet', 'shower', 'thunderstorm', 'damp', 'drizzly']
dry = ['cloudy', 'sunny', 'clear', 'dry', 'overcast', 'fine', 'dried', 'clouds', 'hazy', 'cloud']
temp = ['cool', 'cold', 'hot', 'warm', 'temperatures', 'air']
windy = ['windy']
for i in range(1084):
    weather_main = ''
    is_wet = False
    is_dry = False
    is_temp = False
    is_windy = False
    is_changing = False
    link = races['url'][i]
    response = requests.get(url=link)
    soup = BeautifulSoup(response.content, 'html.parser')
    is_weather_10 = str(soup.select('#mw-content-text > div.mw-parser-output > table.infobox.vevent > tbody > tr:nth-child(10) > th')).split('>')[1].split('<')[0]
    is_weather_11 = str(soup.select('#mw-content-text > div.mw-parser-output > table.infobox.vevent > tbody > tr:nth-child(11) > th')).split('>')[1].split('<')[0]
    is_weather_12 = str(soup.select('#mw-content-text > div.mw-parser-output > table.infobox.vevent > tbody > tr:nth-child(12) > th')).split('>')[1].split('<')[0]
    if is_weather_11 == 'Weather':
        weather = str(soup.select('#mw-content-text > div.mw-parser-output > table.infobox.vevent > tbody > tr:nth-child(11) > td')).split('>')[1].split('<')[0].split('\n')[1]
        if weather == '':
            weather = str(soup.select('#mw-content-text > div.mw-parser-output > table.infobox.vevent > tbody > tr:nth-child(11) > td > p'))
        all_weather.append(weather)
    elif is_weather_12 == 'Weather':
        weather = str(soup.select('#mw-content-text > div.mw-parser-output > table.infobox.vevent > tbody > tr:nth-child(12) > td')).split('>')[1].split('<')[0].split('\n')[1]
        if weather == '':
            weather = str(soup.select('#mw-content-text > div.mw-parser-output > table.infobox.vevent > tbody > tr:nth-child(12) > td > p'))
        all_weather.append(weather)
    elif is_weather_10 == 'Weather':
        weather = str(soup.select('#mw-content-text > div.mw-parser-output > table.infobox.vevent > tbody > tr:nth-child(10) > td')).split('>')[1].split('<')[0].split('\n')[1]
        all_weather.append(weather)
    else:
        all_weather.append(np.nan)
        weather = np.nan
    if weather != np.nan:
        weather = str(weather).lower()
    for j in wet:
        if weather.find(j) != -1:
            is_wet = True
    for j in dry:
        if weather.find(j) != -1:
            is_dry = True
            if is_wet == True:
                is_changing = True
                weather_main = 'changing'
            else:
                weather_main = 'dry'
    for j in temp:
        if weather.find(j) != -1:
            is_temp = True
            if is_dry == False and is_wet == False:
                is_dry = True
                weather_main = 'dry'
    for j in windy:
        if weather.find(j) != -1:
            is_windy = True
            if is_wet == False and is_dry == False:
                is_dry = True
                weather_main = 'dry'
    if is_wet == True and is_changing == False:
        weather_main = 'wet'
    if weather_main == '':
        weather_main = weather
    all_wind.append(is_windy)
    all_weather_main.append(weather_main)
print(all_weather_main)

['dry', 'changing', 'wet', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'changing', 'dry', 'changing', 'changing', 'dry', 'dry', 'dry', 'changing', 'changing', 'dry', 'dry', 'dry', 'changing', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'changing', 'dry', 'dry', 'dry', 'dry', 'wet', 'wet', 'dry', 'dry', 'dry', 'dry', 'dry', 'nan', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'wet', 'dry', 'dry', 'changing', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'changing', 'dry', 'dry', 'dry', 'changing', 'dry', 'dry', 'dry', 'dry', 'changing', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'changing', 'dry', 'dry', 'changing', 'changing', 'dry', 'wet', 'dry', 'dry', 'changing', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'wet', 'dry', 'dry', 'dry', 'dry', 'changing', 'dry', 'dry', 'dry', 'dry

Соединим всю полученную информацию в одну таблицу

In [8]:
weather_df = pd.DataFrame(all_weather).rename({0: 'weather'}, axis='columns')
weather_main_df = pd.DataFrame(all_weather_main).rename({0: 'weather_main'}, axis='columns')
windy_df = pd.DataFrame(all_wind).rename({0: 'wind'}, axis='columns')
w_ = weather_main_df.join(weather_df)
w = w_.join(windy_df)
w

,weather_main,weather,wind
0,dry,Sunny with temperatures reaching up to 27 °C (...,False
1,changing,"Dry start, with heavy rain and thunderstorm/",False
2,wet,Rain,False
3,dry,Sunny,False
4,dry,"Warm, Sunny",False
...,...,...,...
1079,dry,Clear,False
1080,dry,Clear,False
1081,dry,Sunny,False
1082,dry,Partly cloudy,False


In [10]:
w.loc[w['wind'] == True]

,weather_main,weather,wind
37,dry,"Sunny, dry, windy",True
246,dry,"Warm, cloudy, windy",True
285,dry,"Dry, sunny, windy",True
301,dry,"Dry, sunny, windy",True
706,wet,"Rainy, windy, cold",True
727,dry,"Hot, dry and windy",True
736,dry,"Dry, windy",True
793,dry,Cool and windy,True
1044,dry,Windy and partly cloudy. Ambient temperature: ...,True


Как видно выше, среди полученных значений очень мало гонок, в которых среди информации о погоде упоминалось слово "windy", поэтому в дальнейшем мы не учитывали показатель wind

In [7]:
w.to_csv('weather.csv')

Мы сохранили полученную таблицу на github, чтобы в дальнейшем нам не пришлось еще раз загружать код